In [ ]:
import torch


def TrainNetwork(num_epochs):
    """
    Starts the training process of the model.
    Parameters:
        num_epochs (int): Amount of epochs for training the model
    """
    
    data = DataLoader(train_files_path, target_files_path, category_list, split_size, batch_size, train_size)
    
    for epoch in range(num_epochs):
        model.train()
        
        # if there are still unsused images in the image folder, load a new set of batches
        if len(data.train_files) > 0:
            print("LOADING NEW BATCHES")
            print("")
            data.LoadData()
        else: # if all images have been used, load a new instance and repeat the process
            print("DATA IS BEING LOADED FOR A NEW EPOCH")
            print("")
            data = DataLoader(train_files_path, target_files_path, category_list, split_size, batch_size, train_size)
            data.LoadFiles()
            data.LoadData()
        
        for batch_idx, (train_data, target_data) in enumerate(data.train_data):
            train_data = train_data.to(device)
            target_data = target_data.to(device)
            
            predictions = model(train_data)
            loss = YOLO_Loss(predictions, target_data, split_size, num_boxes, num_classes, lambda_coord, lambda_noobj)
            
            optimizer.zero_grad()
            loss.backward()
            
            optimizer.step()
            
            print('Train Epoch: {} of {} [Batch: {}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, num_epochs, batch_idx, len(data.train_data),
                batch_idx / len(train_data) * 100., loss.data.item()))


# Dataset parameters
train_files_path = "C:/Users/alens/Desktop/Real-time-Object-Detection-for-Autonomous-Driving-using-Deep-Learning/YOLO v1/bdd100k/images/100k/val/"
target_files_path = "C:/Users/alens/Desktop/Real-time-Object-Detection-for-Autonomous-Driving-using-Deep-Learning/YOLO v1/bdd100k_labels_release/bdd100k/labels/det_v2_val_release.json"
category_list = ["other vehicle", "pedestrian", "traffic light", "traffic sign", "truck", "train", "other person", "bus", "car", "rider", "motorcycle", "bicycle", "trailer"]


# Hyperparameters
learning_rate = 0.001
split_size = 7
num_boxes = 2
num_classes = len(category_list)
lambda_coord = 5
lambda_noobj = 5
batch_size = 64
num_epochs = 10
train_size = 5000


# Set device
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

# Initialize model
model = YOLOv1(split_size, num_boxes, num_classes).to(device)

# Define the learning method as stochastic gradient descent
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

# Start the training process
TrainNetwork(num_epochs)